# Chapter 4. Structured API Overview

There are three core types of distributed collection APIs
- Datasets
- DataFrames
- SQL tables and views

Majority of the Structured APIs apply to both computation
- `batch` 
- `streaming`

### Note

Before proceeding, let’s review the fundamental concepts and definitions that we covered in Part I.
Spark is a distributed programming model in which the user specifies transformations. Multiple
transformations build up a directed acyclic graph of instructions. An action begins the process of
executing that graph of instructions, as a single job, by breaking it down into stages and tasks to
execute across the cluster. The logical structures that we manipulate with transformations and actions
are DataFrames and Datasets. To create a new DataFrame or Dataset, you call a transformation. To
start computation or convert to native language types, you call an action.

## DataFrames and Datasets
DataFrames and Datasets are (distributed) table-like collections with well-defined rows and
columns. Each column must have the same number of rows as all the other columns (although
you can use null to specify the absence of a value) and each column has type information that
must be consistent for every row in the collection. To Spark, DataFrames and Datasets represent
immutable, lazily evaluated plans that specify what operations to apply to data residing at a
location to generate some output. When we perform an action on a DataFrame, we instruct Spark
to perform the actual transformations and return the result. These represent plans of how to
manipulate rows and columns to compute the user’s desired result.

### NOTE
Tables and views are basically the same thing as DataFrames. We just execute SQL against them
instead of DataFrame code. We cover all of this in Chapter 10, which focuses specifically on Spark
SQL.

### Schemas
A schema defines the **column names** and **types** of a DataFrame. You can define schemas
manually or read a schema from a data source (often called *schema on read*). Schemas consist of
types, meaning that you need a way of specifying what lies where.

### Overview of Structured Spark Types
Spark is effectively a programming language of its own. Internally, Spark uses an engine called
**Catalyst** that maintains its own type information through the planning and processing of work. In
doing so, this opens up a wide variety of execution optimizations that make significant
differences. Spark types map directly to the different language APIs that Spark maintains and
there exists a lookup table for each of these in Scala, Java, Python, SQL, and R. Even if we use
Spark’s Structured APIs from Python or R, the majority of our manipulations will operate strictly
on Spark types, not Python types. For example, the following code does not perform addition in
Scala or Python; it actually performs addition *purely* in Spark:

In [2]:
# set up spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master('local[4]')\
    .appName('Cha4')\
    .getOrCreate()

In [5]:
# This addition operation happens because Spark will convert an expression written in an input
# language to Spark’s internal Catalyst representation of that same type information.
df = spark.range(500).toDF('number')
df.select(df["number"] + 10)

DataFrame[(number + 10): bigint]

### DataFrames Versus Datasets
- DataFrames : untyped (only check types at *runtime*)
- Datasets : typed ( checkt types at *compile time*) 
    - only available to **Java Virtual Machine (JVM)**– based languages (Scala / Java)

**Scala**: To Spark , DataFrames = Datasets of Type `Row`. 
The `“Row”` type is Spark’s internal representation of its optimized
in-memory format for computation. This format makes for highly specialized and efficient
computation because rather than using JVM types, which can cause high garbage-collection and
object instantiation costs, Spark can operate on its own internal format without incurring any of
those costs.   
**Python or R**: no Dataset: everything is a DataFrame -> always operate on that optimized format.

### NOTE
The internal **Catalyst** format is well covered in numerous Spark presentations.  

Talks by [Josh Rosen](https://youtu.be/5ajs8EIPWGI) and [Herman van Hovell](https://youtu.be/GDeePbbCz2g)

Understanding **DataFrames**, **Spark Types**, and **Schemas** takes some time to digest.   
What you need to know is that when you’re using **DataFrames**, you’re taking advantage of Spark’s **optimized internal format**.   
This format applies the same efficiency gains to all of Spark’s language APIs. If you need strict compile-time checking, read Chapter 11 to learn more about it.

## Columns (cha 5)
Represent types:
- *simple type* : `integer`, `string`
- *complex type* : `array`, `map`, `null`

## Rows 
Each record in DataFrame = type `Row`  
Created from 
- SQL
- RDDs
- data sources
- manually from scratch


In [12]:
spark.range(2) .collect() # an array of `Row` objects

[Row(id=0), Row(id=1)]

## Spark Types

Here just include types for reference. For other languages (Java, Scala), please refer to p58- Spark Types.

In [13]:
# python
from pyspark.sql.types import *
b = ByteType()

Complete reference table. Further documentation [here](http://bit.ly/2EdflXW)

![](images/ref1.png)
![](images/ref2.png)
![](images/ref3.png)

## Overview of Structured API Execution
Execution steps:
1. Write DataFrame/Dataset/SQL Code.
2. If valid code, Spark converts this to a Logical Plan.
3. Spark transforms this Logical Plan to a Physical Plan, checking for optimizations along
the way.
4. Spark then executes this Physical Plan (RDD manipulations) on the cluster.

![](images/exec.png)



### Logical Planning
![](images/log.png)

**logical plan** : convert the user’s set of expressions into the most optimized version. 

**unresolved logical plan** : unresolved because although your code might be valid, the tables or columns that it refers to might or might
not exist. Spark uses the catalog, a repository of all table and DataFrame information, to resolve
columns and tables in the *analyzer*. The analyzer might reject the unresolved logical plan if the
required table or column name does not exist in the catalog. 

**Logical Optimization** : If the analyzer can resolve it, the
result is passed through the Catalyst Optimizer, a collection of rules that attempt to optimize the
logical plan by pushing down predicates or selections. Packages can extend the Catalyst to
include their own rules for domain-specific optimizations.

### Physical planning 
After successfully creating an optimized logical plan, Spark then begins the physical planning
process. The physical plan, often called a **Spark plan**, specifies how the logical plan will execute
on the cluster by generating different physical execution strategies and comparing them through
a cost model, as depicted in Figure 4-3. An example of the cost comparison might be choosing
how to perform a given join by looking at the physical attributes of a given table (how big the
table is or how big its partitions are).

![](images/phy.png)

**Physical planning** ----->>>> a series of *RDDs* and *transformations.*

### Execution
Upon selecting a physical plan, Spark runs all of this code over RDDs, the lower-level
programming interface of Spark ( Part III). Spark performs further
*optimizations at runtime*, generating native Java bytecode that can remove entire tasks or stages
during execution. Finally the result is returned to the user.